In [ ]:
%pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
import random
import json
import pandas as pd
from datetime import datetime
from typing import Tuple, Dict, List


class FatigueDatasetGenerator:
    def __init__(self, samples_per_case: int = 300):
        self.samples_per_case = samples_per_case
        self.feature_ranges = self.define_feature_ranges()
        self.intervention_lookup = self.define_intervention_lookup()

    def define_feature_ranges(self) -> Dict[str, Dict[str, Tuple[float, float]]]:
        """Define realistic value ranges for each feature by fatigue class (Low/Moderate/High)"""
        return {
            "Blink Rate": {"Low": (10, 15), "Moderate": (15, 18), "High": (18, 30)},
            "Yawning Rate": {"Low": (0, 0), "Moderate": (1, 1), "High": (2, 5)},
            "PERCLOS": {"Low": (0, 15), "Moderate": (15, 30), "High": (30, 60)},
            "SDLP": {"Low": (0, 20), "Moderate": (20, 35), "High": (35, 50)},  # cm
            "Steering Entropy": {"Low": (0, 1), "Moderate": (1, 3), "High": (3, 5)},
            "Lane Keeping Ratio": {"Low": (0, 0.7), "Moderate": (0.7, 0.85), "High": (0.85, 1)},
            "Lane Departure Frequency": {"Low": (0, 2.5), "Moderate": (2.5, 3.5), "High": (3.5, 6)},
            "SRR": {"Low": (0, 15), "Moderate": (15, 25), "High": (25, 35)},
            "SAV": {"Low": (1, 4), "Moderate": (5, 9), "High": (10, 15)}
        }

    def define_intervention_lookup(self) -> Dict[Tuple[str, str, str], Dict[str, str]]:
        """Define intervention rules based on CF, SF, LF fatigue levels"""
        return {
            ("Low", "Low", "Low"): {
                "fan": "off", "music": "off", "vibration": "off",
                "reason": "No signs of fatigue detected. No intervention required."
            },
            ("Moderate", "Low", "Low"): {
                "fan": "level 2", "music": "off", "vibration": "off",
                "reason": "Moderate visual fatigue may impair focus. Increased airflow helps maintain alertness without overstimulation."
            },
            ("High", "Low", "Low"): {
                "fan": "level 3", "music": "beep", "vibration": "Vibrate",
                "reason": "Severe visual fatigue threatens awareness. Multi-modal alerts counter visual disengagement effectively."
            },
            ("Low", "Moderate", "Moderate"): {
                "fan": "level 2", "music": "off", "vibration": "off",
                "reason": "Motor and lane variations suggest early fatigue. Moderate airflow stabilizes driver alertness."
            },
            ("Moderate", "Moderate", "Moderate"): {
                "fan": "level 2", "music": "beep", "vibration": "off",
                "reason": "Combined visual and control fatigue detected. Fan and beep boost sensory engagement without physical feedback."
            },
            ("High", "Moderate", "Moderate"): {
                "fan": "level 3", "music": "beep", "vibration": "Vibrate",
                "reason": "High fatigue across systems impairs control. Full intervention improves driver responsiveness and safety."
            },
            ("Low", "High", "High"): {
                "fan": "level 3", "music": "beep", "vibration": "Vibrate",
                "reason": "Physical and lane instability despite visual alertness. Tactile and auditory cues reinforce driver control."
            },
            ("Moderate", "High", "High"): {
                "fan": "level 3", "music": "beep", "vibration": "Vibrate",
                "reason": "Motor and lane fatigue with visual strain detected. Strong multi-sensory cues are required immediately."
            },
            ("High", "High", "High"): {
                "fan": "level 3", "music": "beep", "vibration": "Vibrate",
                "reason": "Critical fatigue in all systems detected. Immediate and full intervention needed to ensure driver safety."
            }
        }

    def sample_feature(self, level: str, feature: str) -> float:
        """Sample a value for a feature based on its fatigue class"""
        low, high = self.feature_ranges[feature][level]
        return round(random.uniform(low, high), 2)

    def generate_sample(self, cf: str, sf: str, lf: str, sample_id: int) -> Dict:
        """Generate one full synthetic sample"""
        timestamp = datetime.utcnow().isoformat()
        features = {
            "Blink Rate": self.sample_feature(cf, "Blink Rate"),
            "Yawning Rate": self.sample_feature(cf, "Yawning Rate"),
            "PERCLOS": self.sample_feature(cf, "PERCLOS"),
            "SDLP": self.sample_feature(lf, "SDLP"),
            "Steering Entropy": self.sample_feature(sf, "Steering Entropy"),
            "Lane Keeping Ratio": self.sample_feature(lf, "Lane Keeping Ratio"),
            "Lane Departure Frequency": self.sample_feature(lf, "Lane Departure Frequency"),
            "SRR": self.sample_feature(sf, "SRR"),
            "SAV": self.sample_feature(sf, "SAV")
        }

        intervention = self.intervention_lookup.get((cf, sf, lf))
        if not intervention:
            return None

        return {
            "ID": sample_id,
            "timestamp": timestamp,
            "features": features,
            "CF": cf,
            "SF": sf,
            "LF": lf,
            "intervention": intervention
        }

    def generate_dataset(self) -> Tuple[List[Dict], pd.DataFrame]:
        """Generate full dataset as JSON list and DataFrame"""
        dataset = []
        records = []
        sample_id = 1
        fatigue_levels = ["Low", "Moderate", "High"]

        for cf in fatigue_levels:
            for sf in fatigue_levels:
                for lf in fatigue_levels:
                    combo = (cf, sf, lf)
                    if combo not in self.intervention_lookup:
                        continue
                    for _ in range(self.samples_per_case):
                        entry = self.generate_sample(cf, sf, lf, sample_id)
                        if entry is None:
                            continue
                        dataset.append(entry)

                        # Flatten for CSV
                        flat = {
                            "ID": entry["ID"],
                            "timestamp": entry["timestamp"],
                            **entry["features"],
                            "CF": cf,
                            "SF": sf,
                            "LF": lf,
                            "fan": entry["intervention"]["fan"],
                            "music": entry["intervention"]["music"],
                            "vibration": entry["intervention"]["vibration"],
                            "reason": entry["intervention"]["reason"]
                        }
                        records.append(flat)
                        sample_id += 1

        return dataset, pd.DataFrame(records)

    def save(self, dataset: List[Dict], df: pd.DataFrame, json_path: str, csv_path: str):
        """Save dataset to JSON and CSV files"""
        with open(json_path, "w") as f_json:
            json.dump(dataset, f_json, indent=2)
        df.to_csv(csv_path, index=False)


# === Usage ===
if __name__ == "__main__":
    generator = FatigueDatasetGenerator(samples_per_case=300)
    dataset_json, dataset_df = generator.generate_dataset()
    generator.save(dataset_json, dataset_df,
                   "synthetic_fatigue_dataset.json",
                   "synthetic_fatigue_dataset.csv")
    print(f"✅ Dataset generated with {len(dataset_json)} samples.")


✅ Dataset generated with 2700 samples.


In [ ]:
dataset_df.head()

,ID,timestamp,Blink Rate,Yawning Rate,PERCLOS,SDLP,Steering Entropy,Lane Keeping Ratio,Lane Departure Frequency,SRR,SAV,CF,SF,LF,fan,music,vibration,reason
0,1,2025-06-12T21:58:14.214482,12.48,0.0,14.01,6.89,0.86,0.24,1.50,10.50,1.90,Low,Low,Low,off,off,off,No signs of fatigue detected. No intervention ...
1,2,2025-06-12T21:58:14.214542,12.28,0.0,12.45,0.15,0.68,0.38,0.50,1.98,1.06,Low,Low,Low,off,off,off,No signs of fatigue detected. No intervention ...
2,3,2025-06-12T21:58:14.214565,12.59,0.0,13.84,5.55,0.45,0.22,1.66,7.13,3.12,Low,Low,Low,off,off,off,No signs of fatigue detected. No intervention ...
3,4,2025-06-12T21:58:14.214586,13.76,0.0,10.08,9.02,0.88,0.39,0.64,2.62,3.95,Low,Low,Low,off,off,off,No signs of fatigue detected. No intervention ...
4,5,2025-06-12T21:58:14.214610,10.96,0.0,14.85,14.51,0.71,0.56,2.02,10.66,3.12,Low,Low,Low,off,off,off,No signs of fatigue detected. No intervention ...


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Load CSV dataset
df = pd.read_csv("synthetic_fatigue_dataset.csv")

# Define features and targets
feature_columns = [
    "Blink Rate", "Yawning Rate", "PERCLOS", "SDLP", "Steering Entropy",
    "Lane Keeping Ratio", "Lane Departure Frequency", "SRR", "SAV"
]

X = df[feature_columns]
y_cf = df["CF"]
y_sf = df["SF"]
y_lf = df["LF"]

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data
X_train, X_test, y_cf_train, y_cf_test = train_test_split(X_scaled, y_cf, test_size=0.2, random_state=42)
_, _, y_sf_train, y_sf_test = train_test_split(X_scaled, y_sf, test_size=0.2, random_state=42)
_, _, y_lf_train, y_lf_test = train_test_split(X_scaled, y_lf, test_size=0.2, random_state=42)

# Train RandomForest classifiers
clf_cf = RandomForestClassifier(random_state=42)
clf_sf = RandomForestClassifier(random_state=42)
clf_lf = RandomForestClassifier(random_state=42)

clf_cf.fit(X_train, y_cf_train)
clf_sf.fit(X_train, y_sf_train)
clf_lf.fit(X_train, y_lf_train)

# Predict on test set
cf_pred = clf_cf.predict(X_test)
sf_pred = clf_sf.predict(X_test)
lf_pred = clf_lf.predict(X_test)

# Print classification reports
print("=== Camera Fatigue (CF) Classification Report ===")
print(classification_report(y_cf_test, cf_pred))
print("\n=== Steering Fatigue (SF) Classification Report ===")
print(classification_report(y_sf_test, sf_pred))
print("\n=== Lane Fatigue (LF) Classification Report ===")
print(classification_report(y_lf_test, lf_pred))


=== Camera Fatigue (CF) Classification Report ===
              precision    recall  f1-score   support

        High       1.00      1.00      1.00       187
         Low       1.00      1.00      1.00       190
    Moderate       1.00      1.00      1.00       163

    accuracy                           1.00       540
   macro avg       1.00      1.00      1.00       540
weighted avg       1.00      1.00      1.00       540


=== Steering Fatigue (SF) Classification Report ===
              precision    recall  f1-score   support

        High       1.00      1.00      1.00       176
         Low       1.00      1.00      1.00       187
    Moderate       1.00      1.00      1.00       177

    accuracy                           1.00       540
   macro avg       1.00      1.00      1.00       540
weighted avg       1.00      1.00      1.00       540


=== Lane Fatigue (LF) Classification Report ===
              precision    recall  f1-score   support

        High       1.00      1.

Prompt Template Generator

In [ ]:
def generate_prompt(row):
    fatigue_section = (
        f"Fatigue Levels:\n"
        f"  - Camera Fatigue: {row['CF']}\n"
        f"  - Steering Fatigue: {row['SF']}\n"
        f"  - Lane Fatigue: {row['LF']}\n"
    )

    feature_section = "Features:\n" + "\n".join([
        f"  - {feat}: {row[feat]}" for feat in [
            "Blink Rate", "Yawning Rate", "PERCLOS", "SDLP", "Steering Entropy",
            "Lane Keeping Ratio", "Lane Departure Frequency", "SRR", "SAV"
        ]
    ])

    assistant_output = (
        f"Intervention:\n"
        f"  - Fan: {row['fan']}\n"
        f"  - Music: {row['music']}\n"
        f"  - Vibration: {row['vibration']}\n"
        f"Reason: {row['reason']}"
    )

    return {
        "messages": [
            {
                "role": "system",
                "content": "You are an intelligent fatigue mitigation assistant. Analyze fatigue levels and sensor features to suggest driver interventions."
            },
            {
                "role": "user",
                "content": f"{fatigue_section}\n{feature_section}"
            },
            {
                "role": "assistant",
                "content": assistant_output
            }
        ]
    }


Save to JSONL for Fine-Tuning

In [ ]:
import json

# Assuming `df` is your DataFrame
with open("llm_fatigue_prompt_data.jsonl", "w") as out_file:
    for _, row in df.iterrows():
        prompt_example = generate_prompt(row)
        out_file.write(json.dumps(prompt_example) + "\n")


Few-Shot LLM Evaluator Script (OpenAI or Huggingface Transformers)

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import torch
import random

# === SETTINGS ===
model_id = "meta-llama/Llama-2-7b-hf"
hf_token = "hf_hExLoiMKvZTzLKJxesZbzIPWTshlSGaCj" # Replace with your actual Hugging Face token
max_new_tokens = 150
num_fewshot = 3
device = "cuda" if torch.cuda.is_available() else "cpu"

# === LOAD DATASET ===
df = pd.read_csv("synthetic_fatigue_dataset.csv")
feature_cols = [
    "Blink Rate", "Yawning Rate", "PERCLOS", "SDLP", "Steering Entropy",
    "Lane Keeping Ratio", "Lane Departure Frequency", "SRR", "SAV"
]

# === PROMPT HELPERS ===
def format_instance(row):
    fatigue_info = (
        f"Camera Fatigue: {row['CF']}\n"
        f"Steering Fatigue: {row['SF']}\n"
        f"Lane Fatigue: {row['LF']}\n"
    )
    features = "\n".join([f"{col}: {row[col]}" for col in feature_cols])
    answer = (
        f"Fan: {row['fan']}\n"
        f"Music: {row['music']}\n"
        f"Vibration: {row['vibration']}\n"
        f"Reason: {row['reason']}"
    )
    return fatigue_info + features, answer

def build_fewshot_prompt(df, test_row):
    fewshots = df.sample(num_fewshot)
    prompt = "You are an AI fatigue assistant. Based on fatigue levels and sensor features, suggest an appropriate intervention and explain the reason.\n\n"

    for _, row in fewshots.iterrows():
        user, answer = format_instance(row)
        prompt += f"User:\n{user}\nAssistant:\n{answer}\n---\n"

    test_input, _ = format_instance(test_row)
    prompt += f"User:\n{test_input}\nAssistant:\n"
    return prompt

# === LOAD MODEL AND TOKENIZER ===
print("🔄 Loading model...")
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=hf_token)

# Configure 8-bit loading
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    use_auth_token=hf_token,
    quantization_config=quantization_config # Add quantization config
)


pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# === RUN INFERENCE ===
test_row = df.sample(1).iloc[0]
prompt = build_fewshot_prompt(df, test_row)

print("\n🧠 Prompt sent to model:\n")
print(prompt)

print("\n🧾 Generating response...\n")
output = pipe(prompt, max_new_tokens=max_new_tokens, do_sample=False)[0]["generated_text"]

print("✅ LLM Response:\n")
print(output[len(prompt):].strip())

🔄 Loading model...


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:902: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



🧠 Prompt sent to model:

You are an AI fatigue assistant. Based on fatigue levels and sensor features, suggest an appropriate intervention and explain the reason.

User:
Camera Fatigue: Moderate
Steering Fatigue: High
Lane Fatigue: High
Blink Rate: 16.24
Yawning Rate: 1.0
PERCLOS: 26.71
SDLP: 41.11
Steering Entropy: 4.47
Lane Keeping Ratio: 0.98
Lane Departure Frequency: 3.94
SRR: 25.13
SAV: 14.39
Assistant:
Fan: level 3
Music: beep
Vibration: Vibrate
Reason: Motor and lane fatigue with visual strain detected. Strong multi-sensory cues are required immediately.
---
User:
Camera Fatigue: Low
Steering Fatigue: Moderate
Lane Fatigue: Moderate
Blink Rate: 11.91
Yawning Rate: 0.0
PERCLOS: 5.04
SDLP: 34.57
Steering Entropy: 2.07
Lane Keeping Ratio: 0.77
Lane Departure Frequency: 2.58
SRR: 23.26
SAV: 7.46
Assistant:
Fan: level 2
Music: off
Vibration: off
Reason: Motor and lane variations suggest early fatigue. Moderate airflow stabilizes driver alertness.
---
User:
Camera Fatigue: High
Steer

Batch Evaluation with Scoring

In [ ]:
import re
from sklearn.metrics import accuracy_score, classification_report

# === FUNCTION: Extract predictions from raw LLM text ===
def extract_prediction(raw_response: str) -> dict:
    """
    Extract fan/music/vibration prediction from raw LLM response text.
    """
    prediction = {
        "fan": "unknown",
        "music": "unknown",
        "vibration": "unknown"
    }

    fan_match = re.search(r"Fan\s*[:\-]\s*(level \d|off)", raw_response, re.IGNORECASE)
    music_match = re.search(r"Music\s*[:\-]\s*(beep|off)", raw_response, re.IGNORECASE)
    vib_match = re.search(r"Vibration\s*[:\-]\s*(Vibrate|off)", raw_response, re.IGNORECASE)

    if fan_match: prediction["fan"] = fan_match.group(1).lower()
    if music_match: prediction["music"] = music_match.group(1).lower()
    if vib_match: prediction["vibration"] = vib_match.group(1).capitalize()

    return prediction

# === FUNCTION: Evaluate multiple test samples ===
def evaluate_llm_batch(df, num_samples=20):
    results = []
    test_rows = df.sample(num_samples, random_state=42)

    for idx, (_, test_row) in enumerate(test_rows.iterrows()):
        prompt = build_fewshot_prompt(df, test_row)
        print(f"\n🔄 [{idx+1}/{num_samples}] Running prompt...")
        try:
            output = pipe(prompt, max_new_tokens=max_new_tokens, do_sample=False)[0]["generated_text"]
            model_output = output[len(prompt):].strip()
            prediction = extract_prediction(model_output)

            results.append({
                "id": test_row["ID"],
                "ground_fan": test_row["fan"].lower(),
                "ground_music": test_row["music"].lower(),
                "ground_vibration": test_row["vibration"].capitalize(),
                "pred_fan": prediction["fan"],
                "pred_music": prediction["music"],
                "pred_vibration": prediction["vibration"],
                "match_fan": prediction["fan"] == test_row["fan"].lower(),
                "match_music": prediction["music"] == test_row["music"].lower(),
                "match_vibration": prediction["vibration"] == test_row["vibration"].capitalize()
            })

        except Exception as e:
            print(f"❌ Error: {e}")
            continue

    return pd.DataFrame(results)

# === RUN BATCH EVALUATION ===
batch_df = evaluate_llm_batch(df, num_samples=20)

# === DISPLAY EVALUATION SUMMARY ===
fan_acc = accuracy_score(batch_df["ground_fan"], batch_df["pred_fan"])
music_acc = accuracy_score(batch_df["ground_music"], batch_df["pred_music"])
vibration_acc = accuracy_score(batch_df["ground_vibration"], batch_df["pred_vibration"])

print("\n=== 🎯 Accuracy Report (LLM vs Ground Truth) ===")
print(f"Fan       Accuracy: {fan_acc * 100:.2f}%")
print(f"Music     Accuracy: {music_acc * 100:.2f}%")
print(f"Vibration Accuracy: {vibration_acc * 100:.2f}%")

# Optional: See full results
batch_df[["id", "ground_fan", "pred_fan", "ground_music", "pred_music", "ground_vibration", "pred_vibration"]]


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



🔄 [1/20] Running prompt...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



🔄 [2/20] Running prompt...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



🔄 [19/20] Running prompt...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



🔄 [20/20] Running prompt...

=== 🎯 Accuracy Report (LLM vs Ground Truth) ===
Fan       Accuracy: 55.00%
Music     Accuracy: 65.00%
Vibration Accuracy: 55.00%


,id,ground_fan,pred_fan,ground_music,pred_music,ground_vibration,pred_vibration
0,1340,level 2,level 4,beep,beep,Off,Vibrate
1,1223,level 2,level 3,beep,beep,Off,Vibrate
2,1107,level 2,level 2,off,off,Off,Off
3,813,level 3,level 2,beep,off,Vibrate,Off
4,1232,level 2,level 2,beep,off,Off,Off
5,566,level 2,level 1,off,beep,Off,Off
6,846,level 3,off,beep,off,Vibrate,Off
7,1005,level 2,level 2,off,off,Off,Off
8,2284,level 3,level 3,beep,off,Vibrate,Off
9,701,level 3,level 1,beep,beep,Vibrate,Off


FINE-TUNING LLaMA IN 3 PHASES

PHASE 1: Generate Fine-Tuning Data

In [ ]:
import json
import pandas as pd

# Load your synthetic dataset
df = pd.read_csv("synthetic_fatigue_dataset.csv")

# Feature columns used in prompts
feature_cols = [
    "Blink Rate", "Yawning Rate", "PERCLOS", "SDLP", "Steering Entropy",
    "Lane Keeping Ratio", "Lane Departure Frequency", "SRR", "SAV"
]

# Format: User & Assistant Text
def format_chat_instance(row):
    fatigue_info = (
        f"Camera Fatigue: {row['CF']}\n"
        f"Steering Fatigue: {row['SF']}\n"
        f"Lane Fatigue: {row['LF']}\n"
    )
    features = "\n".join([f"{col}: {row[col]}" for col in feature_cols])
    input_text = fatigue_info + features

    output_text = (
        f"Fan: {row['fan']}\n"
        f"Music: {row['music']}\n"
        f"Vibration: {row['vibration']}\n"
        f"Reason: {row['reason']}"
    )

    return {
        "messages": [
            {"role": "system", "content": "You are an AI fatigue assistant. Based on fatigue levels and sensor features, decide appropriate intervention and explain why."},
            {"role": "user", "content": input_text},
            {"role": "assistant", "content": output_text}
        ]
    }

# Sample 1000 rows for training
sampled_df = df.sample(1000, random_state=42)

# Save as JSONL
with open("llama_finetune_data.jsonl", "w") as f:
    for _, row in sampled_df.iterrows():
        sample = format_chat_instance(row)
        f.write(json.dumps(sample) + "\n")

print("✅ Phase 1 Complete: Generated llama_finetune_data.jsonl")


✅ Phase 1 Complete: Generated llama_finetune_data.jsonl


In [ ]:
import json

def generate_finetune_prompt(row):
    user_input, assistant_output = format_instance(row)
    return {
        "messages": [
            {
                "role": "system",
                "content": "You are an AI fatigue assistant. Based on fatigue levels and features, decide interventions and explain why."
            },
            {
                "role": "user",
                "content": user_input
            },
            {
                "role": "assistant",
                "content": assistant_output
            }
        ]
    }

# Save to JSONL (e.g. 1000 samples)
subset = df.sample(n=1000, random_state=42)

with open("llama_finetune_data.jsonl", "w") as f:
    for _, row in subset.iterrows():
        example = generate_finetune_prompt(row)
        f.write(json.dumps(example) + "\n")

print("✅ Prompt training data saved as llama_finetune_data.jsonl")


✅ Prompt training data saved as llama_finetune_data.jsonl


PHASE 2: Fine-Tune LLaMA 7B with LoRA (Lightweight Training)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType
import torch

model_id = "meta-llama/Llama-2-7b-hf"
hf_token = "hf_hExLoiMKvZTzLKJxesZbzIPWTshlSGaCj"

# Load model + tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    use_auth_token=hf_token
)

tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=hf_token)
tokenizer.pad_token = tokenizer.eos_token

# Load training data
# Modified to explicitly load from local json file
dataset = load_dataset("json", data_files="llama_finetune_data.jsonl")

# Tokenize messages
def tokenize_fn(example):
    # Ensure the input is a list of messages
    if isinstance(example["messages"], list):
        return tokenizer.apply_chat_template(example["messages"], return_tensors="pt", padding="max_length", truncation=True, max_length=512)
    else:
        # Handle cases where 'messages' might not be a list, or log a warning
        print(f"Warning: 'messages' field is not a list in example: {example}")
        return {"input_ids": [], "attention_mask": []} # Return empty tensors or handle appropriately


tokenized_dataset = dataset.map(tokenize_fn, batched=True)


# Define LoRA config
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

# Inject LoRA into model
model = get_peft_model(model, lora_config)

# Training setup
training_args = TrainingArguments(
    output_dir="./llama-fatigue-finetune",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_dir="./logs",
    save_strategy="epoch",
    fp16=True
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer
)

# Train!
trainer.train()
print("✅ Phase 2 Complete: Model fine-tuned with LoRA")

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 250.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 72.12 MiB is free. Process 73021 has 14.67 GiB memory in use. Of the allocated memory 13.97 GiB is allocated by PyTorch, and 578.80 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_dataset
import torch

# ⛔ DO NOT reload model or tokenizer — you already did it in the earlier cell

# Reload tokenizer just for safety if needed (skip if already available)
# tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=hf_token)

# Load dataset from JSONL
dataset = load_dataset("json", data_files="llama_finetune_data.jsonl")

# Tokenize with chat template
def tokenize(sample):
    return tokenizer.apply_chat_template(
        sample["messages"], return_tensors="pt", padding="max_length",
        truncation=True, max_length=512
    )

tokenized_dataset = dataset.map(tokenize)

# Setup LoRA config
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# Add LoRA adapters to the already-loaded model
model = get_peft_model(model, lora_config)

# Training args
training_args = TrainingArguments(
    output_dir="./llama-fatigue-finetuned",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    num_train_epochs=3,
    fp16=True,
    logging_dir="./logs",
    save_strategy="epoch",
    logging_strategy="epoch",
    report_to="none"
)

# Setup Trainer
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset["train"],
    args=training_args
)

trainer.train()


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.

PHASE 3: Run Inference Using Fine-Tuned Model

In [ ]:
from transformers import pipeline

# Load tokenizer and fine-tuned model
model_dir = "./llama-fatigue-finetune"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForCausalLM.from_pretrained(model_dir)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Create test prompt (example)
prompt = """User:
Camera Fatigue: High
Steering Fatigue: Moderate
Lane Fatigue: Moderate
Blink Rate: 25.6
Yawning Rate: 3
PERCLOS: 41.2
SDLP: 39.6
Steering Entropy: 2.8
Lane Keeping Ratio: 0.71
Lane Departure Frequency: 3.2
SRR: 9.1
SAV: 13.4

Assistant:"""

# Generate response
response = pipe(prompt, max_new_tokens=150, do_sample=False)[0]['generated_text']
print("✅ Inference Result:\n", response[len(prompt):])
